In [1]:
import pandas as pd
import numpy as np
import math
import cmath
import os

In [2]:
# load demand time series
cwd=os.getcwd()
df_demand= pd.read_csv('../../../preprocessing/end_use_demand_model/POPadjusted_demand_00to10.csv')
df_demand['AEST']=pd.to_datetime(df_demand['AEST'])
df_demand=df_demand.set_index('AEST')
df_demand=df_demand.loc[df_demand.index.year<=2009] # drop 2010
df_demand

,NSW,QLD,VIC,SA,TAS,NEM
AEST,,,,,,
2000-01-01 00:00:00,8329.481952,5726.432366,6144.580476,1611.002446,1066.124437,22877.621677
2000-01-01 01:00:00,7377.478612,5548.286510,6388.840815,1463.841815,1049.928623,21828.376375
2000-01-01 02:00:00,6544.479503,5262.845013,5783.082886,1272.836862,1025.744617,19888.988882
2000-01-01 03:00:00,6060.659007,5028.138427,5218.125017,1129.042928,1098.926249,18534.891628
2000-01-01 04:00:00,5916.311304,4896.005504,4916.122613,1065.894377,1134.457590,17928.791389
...,...,...,...,...,...,...
2009-12-31 19:00:00,9486.363689,7428.037753,8138.697008,1866.051805,1201.165076,28120.315331
2009-12-31 20:00:00,9212.556001,7203.645840,7504.186113,1697.477863,1284.477292,26902.343109
2009-12-31 21:00:00,9402.438846,7047.507267,6940.459187,1573.695935,1223.507098,26187.608333


In [3]:
df_demand.describe()

,NSW,QLD,VIC,SA,TAS,NEM
count,87672.000000,87672.000000,87672.000000,87672.000000,87672.000000,87672.000000
mean,10262.628856,7318.156421,7453.892654,1715.852956,1300.178598,28050.709485
std,1630.409374,1091.661674,1003.353939,331.696867,171.433824,3860.516053
min,5916.311304,4538.426892,4722.903711,886.844603,806.537767,17691.397507
25%,9110.063256,6437.391266,6662.735015,1471.087259,1175.899216,25087.974865
50%,10361.673844,7446.589854,7401.704906,1708.734653,1277.920859,28277.330858
75%,11362.392626,8113.916404,8192.643124,1899.831461,1399.464063,30945.156233
max,16515.900731,10841.070069,12817.381362,3699.151861,1982.340431,41732.118584


In [4]:
# population allocation from xenophon et al nature data
zonal_demand_dict={'QLD':{'SEQ' : 0.7244163974432053 , 'SWQ' : 0.07083072372420471 , 'NQ' : 0.1523834093013595, 'CQ' : 0.05236946953123125 } ,
                    'NSW': {'SWNSW' : 0.04263593808258865 , 'CAN' : 0.0874465667998519, 'NCEN' : 0.7917019264492462, 'NNS' : 0.07821556866831332 },
                   'VIC':{ 'LV' : 0.03151580897620921 , 'MEL' : 0.874177320746715 , 'CVIC' : 0.05385411856141513 , 'NVIC': 0.040452751715660526 }, 
                    'SA': {'NSA': 0.1528741873314428,  'ADE' : 0.802293210767033, 'SESA' : 0.044832601901523786},
                   'TAS': {'TAS' : 1.0}  
                  }

In [5]:
# split into zones

df_zonal_demand=pd.DataFrame(index=df_demand.index)

for state, state_demand_info in zonal_demand_dict.items():
    if state!='TAS': # TAS is one zone
        for zone, prop in state_demand_info.items():
            df_zonal_demand[zone]= df_demand[state]*prop

df_zonal_demand['TAS']=df_demand['TAS'].copy()
df_zonal_demand.describe()

,SEQ,SWQ,NQ,CQ,SWNSW,CAN,NCEN,NNS,LV,MEL,CVIC,NVIC,NSA,ADE,SESA,TAS
count,87672.000000,87672.000000,87672.000000,87672.000000,87672.000000,87672.000000,87672.000000,87672.000000,87672.000000,87672.000000,87672.000000,87672.000000,87672.000000,87672.000000,87672.000000,87672.000000
mean,5301.392510,518.350316,1115.165625,383.247970,437.556808,897.431660,8124.943036,802.697352,234.915457,6516.023910,401.422819,301.530469,262.309626,1376.617177,76.926152,1300.178598
std,790.817617,77.323186,166.351128,57.169743,69.514033,142.573702,1290.798242,127.523396,31.621511,877.109258,54.034742,40.588428,50.707889,266.118144,14.870834,171.433824
min,3287.710859,321.460061,691.580963,237.675009,252.247482,517.361112,4683.955057,462.747653,148.846131,4128.655312,254.347816,191.054451,135.575648,711.509404,39.759551,806.537767
25%,4663.351790,455.965082,980.951628,337.122766,388.416093,796.643755,7212.454630,712.548778,209.981484,5824.411844,358.815721,269.525965,224.891269,1180.243321,65.952669,1175.899216
50%,5394.431795,527.447349,1134.736750,389.973960,441.779684,906.092804,8203.357144,810.444212,233.270718,6470.402564,398.612294,299.419331,261.221421,1370.906211,76.607020,1277.920859
75%,5877.854090,574.714571,1236.426244,424.921498,484.446268,993.602226,8995.628131,888.716001,258.197776,7161.822816,441.207574,331.414958,290.435191,1524.221883,85.174388,1399.464063
max,7853.448924,767.880839,1651.999218,567.741089,704.170921,1444.258816,13075.670425,1291.800568,403.950143,11204.664098,690.268775,518.498346,565.504835,2967.804423,165.842603,1982.340431


In [6]:
# check
(df_zonal_demand.sum(axis=1)-df_demand['NEM']).describe()

count    8.767200e+04
mean     3.802134e-12
std      4.460523e-12
min     -2.182787e-11
25%      0.000000e+00
50%      3.637979e-12
75%      7.275958e-12
max      2.182787e-11
dtype: float64

In [7]:
# load network data (from Xenophon et al)
df_nodes=pd.read_csv('network_data/network_nodes.csv', index_col='NODE_ID')
df_links=pd.read_csv('network_data/network_edges.csv',index_col='LINE_ID')
df_links=df_links[['NAME', 'FROM_NODE', 'TO_NODE', 'NUM_LINES', 'LENGTH_KM', 'VOLTAGE_KV']]

In [8]:
# replace node ID with zone name in df_links
dict_node_zone=pd.Series(df_nodes['NEM_ZONE']).to_dict()

df_links=df_links.replace({"FROM_NODE": dict_node_zone})
df_links=df_links.replace({"TO_NODE": dict_node_zone})

df_links=df_links.rename({'FROM_NODE':'FROM_ZONE', 'TO_NODE':'TO_ZONE'}, axis=1) # rename columns node-> zone
df_links.head()


,NAME,FROM_ZONE,TO_ZONE,NUM_LINES,LENGTH_KM,VOLTAGE_KV
LINE_ID,,,,,,
l_1,Callide B Power Station to Calvale,CQ,CQ,1,1.068151,275
l_2,Callide B Power Station to Calvale,CQ,CQ,1,1.178383,275
l_3,Calvale to Callide A Power Station,CQ,CQ,1,1.016681,132
l_4,Braemar to Bulli Creek,SWQ,SWQ,2,91.830573,330
l_5,Moura to Biloela Tee,CQ,CQ,1,47.854105,132


In [9]:
# drop internal lines
df_links = df_links.drop(df_links[df_links.FROM_ZONE==df_links.TO_ZONE].index)
# order FROM_ZONE and TO_ZONE alphabetically for easy bookkeeping
df_links.loc[df_links['FROM_ZONE']>df_links['TO_ZONE'], ['FROM_ZONE', 'TO_ZONE']] = df_links.loc[df_links['FROM_ZONE']>df_links['TO_ZONE'], ['TO_ZONE', 'FROM_ZONE' ]].values
df_links.describe()

,NUM_LINES,LENGTH_KM,VOLTAGE_KV
count,62.000000,62.000000,62.000000
mean,1.161290,111.224310,259.532258
std,0.370801,65.501714,97.556494
min,1.000000,15.277015,110.000000
25%,1.000000,70.415888,220.000000
50%,1.000000,107.925700,275.000000
75%,1.000000,139.152316,330.000000
max,2.000000,315.651935,500.000000


In [10]:
# Voltage-dependent reactance, capacitance and inductance parameters from Xenophon et al
V_XL_dict={110:0.342,132:0.35, 220:0.374, 275:0.386, 330:0.396, 400:0.407, 500:0.420}
V_Cn_dict={110:10.02,132:9.97, 220:9.34, 275:9.05, 330:8.82, 400:8.59, 500:8.31}
V_Lc_dict={110:1.088,132:1.116, 220:1.191, 275:1.203, 330:1.262, 400:1.295, 500:1.338}

In [11]:
# take line capacity to be surge impedence load (SIL)

dict_V_MWlimit={} # voltage-dependent capacity limit in MW
V_list=(list(df_links.VOLTAGE_KV.unique())) # list of voltages
V_list.append(400) # include 400kV for completeness
dict_V_MWlimit_keys=V_list
for v in dict_V_MWlimit_keys:
    XL=V_XL_dict[v]
    Cn=V_Cn_dict[v]
    Lc=V_Lc_dict[v]
    # characteristic impedance Z_c = (R + jL_I) / jL_C
    # SIL= V^2 / |Z_c|
    Zsq= (0.168 + XL*1j) / (2*math.pi*50*Cn*1e-9*1j) # conductor resistance R from Xenophon et al; 50Hz AC system in NEM
    Z=cmath.sqrt(Zsq)
    modZ, argZ = cmath.polar(Z)
    MVlimit=(v**2)/ modZ
    dict_V_MWlimit[v]=MVlimit
print(dict_V_MWlimit)

{330: 276.3889676442977, 275: 196.53342855735855, 220: 129.47973674858662, 500: 600.5875351250976, 132: 49.4908343068981, 110: 34.77855123767018, 400: 396.1066816286745}


In [12]:
# Universal Line Loadability Curve 
# Empirically fitted, see loadability.xlsx
def loadability(length):
    if length<80:
        return 3
    if length > 1300:
        return 0.5
    else:
        return 58.091*(length**(-0.661))

In [13]:
# transmission limit of line [MW] = SIL x loadability
df_links['MWlimit']= (df_links['VOLTAGE_KV'].map(dict_V_MWlimit))*df_links.NUM_LINES*(df_links['LENGTH_KM'].apply(loadability))
df_links.head()

,NAME,FROM_ZONE,TO_ZONE,NUM_LINES,LENGTH_KM,VOLTAGE_KV,MWlimit
LINE_ID,,,,,,,
l_11,Bulli Creek to Dumaresq,NNS,SWQ,2,164.524781,330,1100.856340
l_30,Callide A to Halys,CQ,SWQ,2,315.651935,275,508.860206
l_167,Wodonga Terminal to Jindera,NVIC,SWNSW,1,42.644610,330,829.166903
l_172,Shepparton Terminal Station to Fosterville Tee,CVIC,NVIC,1,93.016255,220,375.918121
l_173,Bendigo Terminal to Ballarat Terminal,CVIC,MEL,1,96.181262,220,367.695114


In [14]:
df_links.tail()

,NAME,FROM_ZONE,TO_ZONE,NUM_LINES,LENGTH_KM,VOLTAGE_KV,MWlimit
LINE_ID,,,,,,,
l_1718,Para to Brinkworth,ADE,NSA,1,33.498863,275,589.600286
l_1853,Eagle Downs to Moranbah,CQ,NQ,1,47.148489,132,148.472503
l_1854,Dysart to Peak Downs Tee,CQ,NQ,1,41.405672,132,148.472503
l_2296,Evans Lane to Shoalhaven,CAN,NCEN,1,59.594539,132,148.472503
l_2298,Tomerong to Shoalhaven,CAN,NCEN,1,15.277015,132,148.472503


In [15]:
# concatenate duplicate with from <-> to reversed
df_links1=df_links[['FROM_ZONE', 'TO_ZONE', 'MWlimit']].reset_index(drop=True)
df_links1=pd.concat([df_links1, df_links1.rename({'FROM_ZONE': 'TO_ZONE','TO_ZONE': 'FROM_ZONE'}, axis=1)],
                    axis=0, ignore_index=True)

df_links1.describe()

,MWlimit
count,124.000000
mean,530.411093
std,337.047742
min,104.335654
25%,296.945006
50%,446.719886
75%,703.443205
max,1508.746436


In [16]:
df_links1.head()

,FROM_ZONE,TO_ZONE,MWlimit
0,NNS,SWQ,1100.856340
1,CQ,SWQ,508.860206
2,NVIC,SWNSW,829.166903
3,CVIC,NVIC,375.918121
4,CVIC,MEL,367.695114


In [17]:
# add AC interconnectors
df_inter=pd.read_csv('network_data/network_ac_interconnector_links.csv', index_col=0)
df_flow=pd.read_csv('network_data/network_ac_interconnector_flow_limits.csv', index_col=0)
df_hvdc=pd.read_csv('network_data/network_hvdc_links.csv',index_col=0)

In [18]:
# replace node number with zone name

# node 204 (Murray) is located in CAN but all power generated is allocated to VIC
dict_node_zone[204]='NVIC'

df_inter=df_inter.replace({"FROM_NODE": dict_node_zone})
df_inter=df_inter.replace({"TO_NODE": dict_node_zone})
df_inter=df_inter.rename({'FROM_NODE':'FROM_ZONE', 'TO_NODE':'TO_ZONE'}, axis=1) # rename columns node-> zone
df_inter=df_inter.reset_index()
df_inter

,INTERCONNECTOR_ID,FROM_ZONE,FROM_REGION,TO_ZONE,TO_REGION,VOLTAGE_KV
0,N-Q-MNSP1,NNS,NSW1,SEQ,QLD1,110
1,NSW1-QLD1,NNS,NSW1,SWQ,QLD1,330
2,V-SA,MEL,VIC1,SESA,SA1,275
3,VIC1-NSW1,NVIC,VIC1,CAN,NSW1,330
4,VIC1-NSW1,NVIC,VIC1,CAN,NSW1,330
5,VIC1-NSW1,NVIC,VIC1,SWNSW,NSW1,330
6,VIC1-NSW1,CVIC,VIC1,SWNSW,NSW1,220
7,VIC1-NSW1,NVIC,VIC1,CAN,NSW1,132


In [19]:
# add flow limit 
df_inter['FORWARD_LIMIT_MW']=np.nan # initialise flow limit columns
df_inter['REVERSE_LIMIT_MW']=np.nan

# for VIC-NSW interconnectors, assume power \prop V^2
v_V2=df_inter.loc[df_inter['INTERCONNECTOR_ID']=='VIC1-NSW1', 'VOLTAGE_KV'].values 
sum_V2=np.dot(v_V2, v_V2).sum()

for idx in df_inter.index:
    inter_ID=df_inter.loc[idx, 'INTERCONNECTOR_ID']
    if inter_ID=='VIC1-NSW1':
        factor=df_inter.loc[idx, 'VOLTAGE_KV']**2 / sum_V2
        df_inter.loc[idx, 'FORWARD_LIMIT_MW']=df_flow.loc[inter_ID, 'FORWARD_LIMIT_MW']*factor
        df_inter.loc[idx, 'REVERSE_LIMIT_MW']=df_flow.loc[inter_ID, 'REVERSE_LIMIT_MW']*factor
    else:
        df_inter.loc[idx, 'FORWARD_LIMIT_MW']=df_flow.loc[inter_ID, 'FORWARD_LIMIT_MW']
        df_inter.loc[idx, 'REVERSE_LIMIT_MW']=df_flow.loc[inter_ID, 'REVERSE_LIMIT_MW']
    

df_inter

,INTERCONNECTOR_ID,FROM_ZONE,FROM_REGION,TO_ZONE,TO_REGION,VOLTAGE_KV,FORWARD_LIMIT_MW,REVERSE_LIMIT_MW
0,N-Q-MNSP1,NNS,NSW1,SEQ,QLD1,110,107.000000,210.000000
1,NSW1-QLD1,NNS,NSW1,SWQ,QLD1,330,600.000000,1078.000000
2,V-SA,MEL,VIC1,SESA,SA1,275,600.000000,500.000000
3,VIC1-NSW1,NVIC,VIC1,CAN,NSW1,330,443.896424,374.537608
4,VIC1-NSW1,NVIC,VIC1,CAN,NSW1,330,443.896424,374.537608
5,VIC1-NSW1,NVIC,VIC1,SWNSW,NSW1,330,443.896424,374.537608
6,VIC1-NSW1,CVIC,VIC1,SWNSW,NSW1,220,197.287300,166.461159
7,VIC1-NSW1,NVIC,VIC1,CAN,NSW1,132,71.023428,59.926017


In [20]:
# HVDC interconnectors

df_hvdc=df_hvdc.replace({"FROM_NODE": dict_node_zone})
df_hvdc=df_hvdc.replace({"TO_NODE": dict_node_zone})
df_hvdc=df_hvdc.rename({'FROM_NODE':'FROM_ZONE', 'TO_NODE':'TO_ZONE'}, axis=1) # rename columns node-> zone
df_hvdc=df_hvdc.reset_index()

df_hvdc

,HVDC_LINK_ID,FROM_ZONE,TO_ZONE,FORWARD_LIMIT_MW,REVERSE_LIMIT_MW,VOLTAGE_KV
0,DIRECTLINK,NNS,NNS,180,180,132
1,T-V-MNSP1,TAS,LV,594,478,400
2,V-S-MNSP1,CVIC,NSA,220,200,220


In [21]:
# DIRECTLINK is part of N-Q-MNSP1 (Terranora), delete
df_hvdc=df_hvdc.drop(0)
df_hvdc=df_hvdc.reset_index()
df_hvdc

,index,HVDC_LINK_ID,FROM_ZONE,TO_ZONE,FORWARD_LIMIT_MW,REVERSE_LIMIT_MW,VOLTAGE_KV
0,1,T-V-MNSP1,TAS,LV,594,478,400
1,2,V-S-MNSP1,CVIC,NSA,220,200,220


In [22]:
# all interconnectors

df_all_interconnectors=pd.concat([
    df_inter[['FROM_ZONE','TO_ZONE','FORWARD_LIMIT_MW']],
    df_inter[['FROM_ZONE','TO_ZONE','REVERSE_LIMIT_MW']].rename({'FROM_ZONE':'TO_ZONE', 
                                                                 'TO_ZONE':'FROM_ZONE', 
                                                                 'REVERSE_LIMIT_MW':'FORWARD_LIMIT_MW'
                                                                }, axis=1), 
    
    df_hvdc[['FROM_ZONE','TO_ZONE','FORWARD_LIMIT_MW']],
    df_hvdc[['FROM_ZONE','TO_ZONE','REVERSE_LIMIT_MW']].rename({'FROM_ZONE':'TO_ZONE', 
                                                                 'TO_ZONE':'FROM_ZONE', 
                                                                 'REVERSE_LIMIT_MW':'FORWARD_LIMIT_MW'
                                                                }, axis=1), 
    
], axis=0, ignore_index=True)


In [23]:
df_all_lines=pd.concat([
    df_links1.rename({'MWlimit': 'FORWARD_LIMIT_MW'}, axis=1),
    df_all_interconnectors
], axis=0, ignore_index=True)
df_all_lines

,FROM_ZONE,TO_ZONE,FORWARD_LIMIT_MW
0,NNS,SWQ,1100.856340
1,CQ,SWQ,508.860206
2,NVIC,SWNSW,829.166903
3,CVIC,NVIC,375.918121
4,CVIC,MEL,367.695114
...,...,...,...
139,CAN,NVIC,59.926017
140,TAS,LV,594.000000
141,CVIC,NSA,220.000000
142,LV,TAS,478.000000


In [24]:
# check for closed loops
for i in df_all_lines.index:
    if df_all_lines.loc[i, 'FROM_ZONE']==df_all_lines.loc[i, 'TO_ZONE']:
        print(i)

In [25]:
# sum export limit for each zone

df_export_lim=df_all_lines.groupby('FROM_ZONE').sum().reset_index()
df_export_lim=df_export_lim.rename({'FROM_ZONE': 'zone', 'FORWARD_LIMIT_MW':'export_limit_MW'}, axis=1)
df_export_lim=df_export_lim.set_index('zone')
df_export_lim

/tmp/ipykernel_49911/2875234082.py:3: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_export_lim=df_all_lines.groupby('FROM_ZONE').sum().reset_index()


,export_limit_MW
zone,
ADE,2959.318439
CAN,8573.171441
CQ,3597.039125
CVIC,1937.778955
LV,7874.293896
MEL,11164.702131
NCEN,6777.402178
NNS,3576.675547
NQ,1798.934509


In [26]:
# find peak demand in each zone
df_max_demand=df_zonal_demand.max()
df_max_demand.name='peak_demand_MW'
df_max_demand

SEQ       7853.448924
SWQ        767.880839
NQ        1651.999218
CQ         567.741089
SWNSW      704.170921
CAN       1444.258816
NCEN     13075.670425
NNS       1291.800568
LV         403.950143
MEL      11204.664098
CVIC       690.268775
NVIC       518.498346
NSA        565.504835
ADE       2967.804423
SESA       165.842603
TAS       1982.340431
Name: peak_demand_MW, dtype: float64

In [27]:
# max NEM demand 
peak_NEM_demand=df_demand['NEM'].max()
peak_NEM_demand

41732.11858361324

In [28]:
# use the same order as in residual demand dataframes
ICM=1
df_raw=pd.read_csv('../../../preprocessing/residual_demand/residual_s%s.csv' %str(int(ICM*10)))
df_raw['AEST']=pd.to_datetime(df_raw['AEST'])
df_raw=df_raw.set_index('AEST')
df_raw.describe()


,NSA_wind,ADE_wind,SESA_wind,NQ_wind,CQ_wind,SWQ_wind,SEQ_wind,NNS_wind,NCEN_wind,SWNSW_wind,...,SEQ_pv,NNS_pv,NCEN_pv,SWNSW_pv,CAN_pv,CVIC_pv,NVIC_pv,MEL_pv,LV_pv,TAS_pv
count,87672.000000,87672.000000,87672.000000,87672.000000,87672.000000,87672.000000,87672.000000,87672.000000,87672.000000,87672.000000,...,87672.000000,87672.000000,87672.000000,87672.000000,87672.000000,87672.000000,87672.000000,87672.000000,87672.000000,87672.000000
mean,0.184985,0.153211,0.189412,0.259803,0.273037,0.300236,0.373341,0.302220,0.261581,0.241841,...,0.448423,0.441335,0.448039,0.444629,0.450583,0.451310,0.458223,0.471582,0.480718,0.488038
std,0.282661,0.330466,0.309647,0.213330,0.236918,0.243225,0.244021,0.262312,0.293881,0.289658,...,0.269137,0.276666,0.271258,0.275971,0.280165,0.275886,0.271921,0.259817,0.250789,0.254574
min,-0.490029,-0.521974,-0.516377,-0.367623,-0.411468,-0.432330,-0.527425,-0.465922,-0.493856,-0.505501,...,-0.305638,-0.343736,-0.344529,-0.376158,-0.406432,-0.376048,-0.380637,-0.412356,-0.378937,-0.420635
25%,-0.045190,-0.135820,-0.067576,0.103695,0.091806,0.115543,0.225266,0.099680,0.020510,0.008485,...,0.244022,0.230043,0.249956,0.243494,0.269220,0.262721,0.287255,0.333441,0.359794,0.391667
50%,0.174060,0.134492,0.193381,0.252846,0.254806,0.290737,0.408425,0.304164,0.265702,0.240410,...,0.542947,0.542451,0.544302,0.544450,0.549592,0.548975,0.551092,0.554553,0.556045,0.563666
75%,0.409855,0.437637,0.442782,0.416920,0.454252,0.494867,0.552677,0.518896,0.509157,0.480440,...,0.646140,0.643861,0.645051,0.644239,0.649174,0.647546,0.649521,0.650415,0.651464,0.657149
max,0.896640,0.973785,0.956736,0.869499,0.890893,0.917156,0.946974,0.911965,0.977314,0.933389,...,0.977755,0.977755,0.977755,0.977755,0.977755,0.977755,0.977755,0.977755,0.977755,0.977755


In [29]:
zone_order=[col.split('_')[0] for col in df_raw.columns[:16]]
zone_order


['NSA',
 'ADE',
 'SESA',
 'NQ',
 'CQ',
 'SWQ',
 'SEQ',
 'NNS',
 'NCEN',
 'SWNSW',
 'CAN',
 'CVIC',
 'NVIC',
 'MEL',
 'LV',
 'TAS']

In [30]:
df_summary=df_export_lim.join(df_max_demand)
df_summary['max_share']=(df_summary['export_limit_MW']+df_summary['peak_demand_MW'])/peak_NEM_demand
df_summary['min_share']=((df_summary['peak_demand_MW']-df_summary['export_limit_MW']).clip(lower=0))/peak_NEM_demand
df_summary=df_summary.loc[zone_order]
df_summary

,export_limit_MW,peak_demand_MW,max_share,min_share
zone,,,,
NSA,2402.130280,565.504835,0.071112,0.000000
ADE,2959.318439,2967.804423,0.142028,0.000203
SESA,2415.645640,165.842603,0.061859,0.000000
NQ,1798.934509,1651.999218,0.082693,0.000000
CQ,3597.039125,567.741089,0.099798,0.000000
SWQ,6224.774630,767.880839,0.167561,0.000000
SEQ,5244.973802,7853.448924,0.313869,0.062505
NNS,3576.675547,1291.800568,0.116660,0.000000
NCEN,6777.402178,13075.670425,0.475726,0.150921


In [31]:
df_summary.to_csv('zonal_constraints.csv')

### zonal constraints in matrix form

In [73]:
# upperbound

M_max=np.hstack([np.identity(16), np.identity(16)])
X_max=df_summary['max_share'].values

print(M_max.shape, X_max.shape)

# lower bound

M_min=-M_max
X_min=-df_summary['min_share'].values
M_min=M_min[np.where(X_min!=0)]
lb_zones=[zone_order[i] for i in np.where(X_min!=0)[0]]
X_min=X_min[np.where(X_min!=0)]
print(lb_zones)
print(M_min.shape, X_min.shape)

(16, 32) (16,)
['ADE', 'SEQ', 'NCEN', 'MEL', 'TAS']
(5, 32) (5,)


In [74]:
M_min

array([[-0., -1., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.,
        -0., -0., -0., -0., -1., -0., -0., -0., -0., -0., -0., -0., -0.,
        -0., -0., -0., -0., -0., -0.],
       [-0., -0., -0., -0., -0., -0., -1., -0., -0., -0., -0., -0., -0.,
        -0., -0., -0., -0., -0., -0., -0., -0., -0., -1., -0., -0., -0.,
        -0., -0., -0., -0., -0., -0.],
       [-0., -0., -0., -0., -0., -0., -0., -0., -1., -0., -0., -0., -0.,
        -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -1., -0.,
        -0., -0., -0., -0., -0., -0.],
       [-0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.,
        -1., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.,
        -0., -0., -0., -1., -0., -0.],
       [-0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.,
        -0., -0., -1., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.,
        -0., -0., -0., -0., -0., -1.]])

In [75]:
X_min

array([-0.00020334, -0.06250522, -0.15092136, -0.00095758, -0.03326791])

In [76]:
Aub_zonal=np.vstack([M_max, M_min])
bub_zonal=np.append(X_max, X_min)

In [77]:
Aub_zonal.shape

(21, 32)

In [78]:
bub_zonal

array([ 7.11115375e-02,  1.42027845e-01,  6.18585476e-02,  8.26925122e-02,
        9.97979579e-02,  1.67560519e-01,  3.13869105e-01,  1.16660172e-01,
        4.75726449e-01,  8.63269163e-02,  2.40041258e-01,  6.29742227e-02,
        1.38674942e-01,  5.36022780e-01,  1.98366254e-01,  6.17351939e-02,
       -2.03344198e-04, -6.25052169e-02, -1.50921364e-01, -9.57582989e-04,
       -3.32679116e-02])

In [79]:
bub_zonal.shape

(21,)

In [80]:
import pickle

file = open('Aub_zonal.pkl', 'wb')

# dump information to that file
pickle.dump(Aub_zonal, file)

# close the file
file.close()

file1 = open('bub_zonal.pkl', 'wb')

# dump information to that file
pickle.dump(bub_zonal, file1)

# close the file
file1.close()

file2 = open('lb_zones.pkl', 'wb')

# dump information to that file
pickle.dump(lb_zones, file2)

# close the file
file2.close()